# Data Extraction

In [1]:
import xarray as xr
import os
import numpy as np
import pandas as pd

# Specify the input and output directories
input_directory = r'D:\Thesis\Thesis Papers\GCM SWAT Integration\Models Dataset\Pr\SSP4.5'
output_directory = r'D:\Thesis\Thesis Papers\GCM SWAT Integration\Models Dataset\Pr\SSP4.5'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get the list of NetCDF files in the input directory
file_list = [f for f in os.listdir(input_directory) if f.endswith('.nc')]

# Specify the list of latitude and longitude pairs
coordinates_list = [(25.62, 40.31, 'A1'),
                    (24.38, 40.31, 'A2'),
                    (24.38, 42.21, 'A3'),
                    (23.13, 42.19, 'A4'),
                    (23.11, 40.79, 'A5'),
                    (23.11, 39.38, 'A6')]

# Create a dictionary to store time series for each coordinate
coordinate_timeseries = {coord: pd.DataFrame() for coord in coordinates_list}

# Loop through each file
for file_name in file_list:
    # Construct the file path
    file_path = os.path.join(input_directory, file_name)

    # Read the NetCDF file using xarray
    ds = xr.open_dataset(file_path)

    # Access the precipitation variable
    pr_variable = ds['pr']

    # Create a new folder for the file's output
    file_output_directory = os.path.join(output_directory, file_name[:-3])
    os.makedirs(file_output_directory, exist_ok=True)

    # Filter the time series from 01/01/2026 to 31/12/2100
    pr_variable_filtered = pr_variable.sel(time=slice('2026-01-01', '2100-12-31'))

    # Create an Excel writer
    excel_file_path = os.path.join(file_output_directory, f'{file_name[:-3]}.xlsx')
    with pd.ExcelWriter(excel_file_path) as writer:

        # Loop through coordinates
        for lat, lon, station_name in coordinates_list:
            # Find the nearest indices in the lat and lon dimensions
            lat_index = np.abs(pr_variable_filtered.lat - lat).argmin()
            lon_index = np.abs(pr_variable_filtered.lon - lon).argmin()

            # Extract the precipitation values at the specified indices
            pr_at_location = pr_variable_filtered[:, lat_index, lon_index]

            # Convert the units to mm/day
            pr_at_location_mm_per_day = pr_at_location * 86400

            # Add the date at the beginning of each text file
            pr_dataframe = pr_at_location_mm_per_day.to_dataframe()
            pr_dataframe.index = pd.date_range(start='2026-01-01', periods=len(pr_dataframe), freq='D')

            # Save the 'pr' variable values to a text file without the '#' in the header
            text_file_path = os.path.join(file_output_directory, f'{station_name}.txt')
            np.savetxt(text_file_path, pr_dataframe['pr'].values, fmt='%.6f', header='20260101', comments='')

            # Save the 'pr' variable values to an Excel sheet with the station name as the sheet name
            pr_dataframe.to_excel(writer, sheet_name=f'station_{station_name}', index=False, header=None)

            # Add the time series to the coordinate_timeseries dictionary
            coordinate_timeseries[(lat, lon, station_name)][file_name[:-3]] = pr_dataframe['pr']

        # Create a new dataframe for average values
        average_dataframe = pd.DataFrame()

        # Loop through coordinates again to calculate the average
        for lat, lon, _ in coordinates_list:
            # Find the nearest indices in the lat and lon dimensions
            lat_index = np.abs(pr_variable_filtered.lat - lat).argmin()
            lon_index = np.abs(pr_variable_filtered.lon - lon).argmin()

            # Extract the precipitation values at the specified indices
            pr_at_location = pr_variable_filtered[:, lat_index, lon_index]

            # Convert the units to mm/day
            pr_at_location_mm_per_day = pr_at_location * 86400

            # Add the precipitation values to the average dataframe
            average_dataframe[f'Lat={lat}, Lon={lon}'] = pr_at_location_mm_per_day

        # Calculate the average of all time series
        average_dataframe['Average'] = average_dataframe.mean(axis=1)

        # Set the index as date range
        average_dataframe.index = pd.date_range(start='2026-01-01', periods=len(average_dataframe), freq='D')

        # Save the average values to a separate sheet in the Excel file
        average_dataframe.to_excel(writer, sheet_name='Average', index=True)

    # Close the dataset when you are done with the current file
    ds.close()

# Create a directory for the coordinate time series text files
coordinate_timeseries_txt_directory = os.path.join(output_directory, 'coordinate_timeseries_txt')
os.makedirs(coordinate_timeseries_txt_directory, exist_ok=True)

# Create an Excel file for the coordinate time series
coordinate_timeseries_file_path = os.path.join(output_directory, 'coordinate_timeseries.xlsx')
with pd.ExcelWriter(coordinate_timeseries_file_path) as writer:
    for coord, timeseries_df in coordinate_timeseries.items():
        # Add the average column
        timeseries_df['Average'] = timeseries_df.mean(axis=1)

        # Save the time series to a sheet in the Excel file
        sheet_name = f"{coord[2]}"
        timeseries_df.to_excel(writer, sheet_name=sheet_name)

        # Save the Average time series to a text file
        average_timeseries = timeseries_df['Average']
        text_file_path = os.path.join(coordinate_timeseries_txt_directory, f'{coord[2]}.txt')
        np.savetxt(text_file_path, average_timeseries.values, fmt='%.6f', header='20260101', comments='')

## Historical

In [ ]:
import xarray as xr
import os
import numpy as np
import pandas as pd

# Specify the input and output directories
input_directory = r'D:\GCM\Dataset\GFDL'
output_directory = r'D:\GCM\Dataset\GFDL\output'

# Create the output directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Get the list of NetCDF files in the input directory
file_list = [f for f in os.listdir(input_directory) if f.endswith('.nc')]

# Specify the list of latitude and longitude pairs
coordinates_list = [(25.62, 40.31, 'A1'),
                    (24.38, 40.31, 'A2'),
                    (24.38, 42.21, 'A3'),
                    (23.13, 42.19, 'A4'),
                    (23.11, 40.79, 'A5'),
                    (23.11, 39.38, 'A6')]

# Create a dictionary to store time series for each coordinate
coordinate_timeseries = {coord: pd.DataFrame() for coord in coordinates_list}

# Loop through each file
for file_name in file_list:
    # Construct the file path
    file_path = os.path.join(input_directory, file_name)

    # Read the NetCDF file using xarray
    ds = xr.open_dataset(file_path)

    # Access the precipitation variable
    pr_variable = ds['pr']

    # Create a new folder for the file's output
    file_output_directory = os.path.join(output_directory, file_name[:-3])
    os.makedirs(file_output_directory, exist_ok=True)

    # Filter the time series from 01/01/1979 to 31/12/2010
    pr_variable_filtered = pr_variable.sel(time=slice('1979-01-01', '2005-12-31'))

    # Create an Excel writer
    excel_file_path = os.path.join(file_output_directory, f'{file_name[:-3]}.xlsx')
    with pd.ExcelWriter(excel_file_path) as writer:

        # Loop through coordinates
        for lat, lon, station_name in coordinates_list:
            # Find the nearest indices in the lat and lon dimensions
            lat_index = np.abs(pr_variable_filtered.lat - lat).argmin()
            lon_index = np.abs(pr_variable_filtered.lon - lon).argmin()

            # Extract the precipitation values at the specified indices
            pr_at_location = pr_variable_filtered[:, lat_index, lon_index]

            # Convert the units to mm/day
            pr_at_location_mm_per_day = pr_at_location * 86400

            # Add the date at the beginning of each text file
            pr_dataframe = pr_at_location_mm_per_day.to_dataframe()
            pr_dataframe.index = pd.date_range(start='1979-01-01', periods=len(pr_dataframe), freq='D')

            # Save the 'pr' variable values to a text file without the '#' in the header
            text_file_path = os.path.join(file_output_directory, f'{station_name}.txt')
            np.savetxt(text_file_path, pr_dataframe['pr'].values, fmt='%.6f', header='19790101', comments='')

            # Save the 'pr' variable values to an Excel sheet with the station name as the sheet name
            pr_dataframe.to_excel(writer, sheet_name=f'station_{station_name}', index=False, header=None)

            # Add the time series to the coordinate_timeseries dictionary
            coordinate_timeseries[(lat, lon, station_name)][file_name[:-3]] = pr_dataframe['pr']

        # Create a new dataframe for average values
        average_dataframe = pd.DataFrame()

        # Loop through coordinates again to calculate the average
        for lat, lon, _ in coordinates_list:
            # Find the nearest indices in the lat and lon dimensions
            lat_index = np.abs(pr_variable_filtered.lat - lat).argmin()
            lon_index = np.abs(pr_variable_filtered.lon - lon).argmin()

            # Extract the precipitation values at the specified indices
            pr_at_location = pr_variable_filtered[:, lat_index, lon_index]

            # Convert the units to mm/day
            pr_at_location_mm_per_day = pr_at_location * 86400

            # Add the precipitation values to the average dataframe
            average_dataframe[f'Lat={lat}, Lon={lon}'] = pr_at_location_mm_per_day

        # Calculate the average of all time series
        average_dataframe['Average'] = average_dataframe.mean(axis=1)

        # Set the index as date range
        average_dataframe.index = pd.date_range(start='1979-01-01', periods=len(average_dataframe), freq='D')

        # Save the average values to a separate sheet in the Excel file
        average_dataframe.to_excel(writer, sheet_name='Average', index=True)

    # Close the dataset when you are done with the current file
    ds.close()

# Create a directory for the coordinate time series text files
coordinate_timeseries_txt_directory = os.path.join(output_directory, 'coordinate_timeseries_txt')
os.makedirs(coordinate_timeseries_txt_directory, exist_ok=True)

# Create an Excel file for the coordinate time series
coordinate_timeseries_file_path = os.path.join(output_directory, 'coordinate_timeseries.xlsx')
with pd.ExcelWriter(coordinate_timeseries_file_path) as writer:
    for coord, timeseries_df in coordinate_timeseries.items():
        # Add the average column
        timeseries_df['Average'] = timeseries_df.mean(axis=1)

        # Save the time series to a sheet in the Excel file
        sheet_name = f"{coord[2]}"
        timeseries_df.to_excel(writer, sheet_name=sheet_name)

        # Save the Average time series to a text file
        average_timeseries = timeseries_df['Average']
        text_file_path = os.path.join(coordinate_timeseries_txt_directory, f'{coord[2]}.txt')
        np.savetxt(text_file_path, average_timeseries.values, fmt='%.6f', header='19790101', comments='')